### Apredizaje supervisado
#### Autor: Rahimi Vilchez, Aiad

En este trabajo se aplicarán algunas técnicas aprendidas para el aprendizaje automático supervisado.

Se utilizará un dataset de airbnb donde se buscará predecir el precio de las noches de cada hogar en función de algunas variables.

#### Variables:
- Unnamed: 0: Variable de contador de registros (no será utilizada en nuetro análisis)
- neighbourhood: Barrio donde se encuentra ubicado la casa.
- room_type: Tipo de hogar en alquiler.
- price: Precio por noche.
- minimum_nights: Cantidad minimas de noches que pueden alquilar el hogar.
- number_of_reviews: Cantidad de reviews
- reviews_per_month: Tasa de reviews que tienen por mes
- calculated_host_listings_count: Cantidad de hogares ofrecidos por el host.
- availability_365: Cantidad de dias que se encuentra en alquiler el hogar

### Objetivos principales:
- resolver los requisitos que se detallan en las proximas celdas

In [39]:
## Importo librerías de maniupulación de datos
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [40]:
# Importo librerías de Aprendizaje automático
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR, LinearSVR
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import GridSearchCV

##### **Objetivo 1**
Lo primero que deberan hacer será importar
- el dataset bajo en nombre "london"
- corroborar que lo hayamos cargado correctamente.

In [41]:
# Cargo london y corroboro
london = pd.read_csv('TP2_airbnb_london.csv')
london.tail(10)

,Unnamed: 0,neighbourhood,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
51817,74704,Tower Hamlets,Entire home/apt,90,3,1,1.0,1,195
51818,74729,Lambeth,Entire home/apt,188,2,1,1.0,1,294
51819,74741,Westminster,Entire home/apt,60,1,1,1.0,1,82
51820,74754,Hammersmith and Fulham,Entire home/apt,120,3,1,1.0,1,26
51821,74785,Brent,Entire home/apt,150,1,1,1.0,8,327
51822,74816,Brent,Entire home/apt,60,5,1,1.0,1,7
51823,74927,Lewisham,Entire home/apt,30,2,1,1.0,1,26
51824,75003,Westminster,Entire home/apt,280,3,1,1.0,14,352
51825,75054,Tower Hamlets,Private room,35,2,1,1.0,1,13
51826,75192,Westminster,Private room,66,2,1,1.0,1,140


##### **Objetivo 2**
Ahora imprimir también las dimensiones del dataset y eliminar la variable "Unnamed:0" del dataset.
Volver a guardar el dataset sin esta variable bajo el nombre "london" 

In [42]:
# Imprimo dimensiones
dimensiones = london.shape
print(f"Las dimensiones del dataset london son {dimensiones}")
london = london.drop(['Unnamed: 0'], axis = 1)

Las dimensiones del dataset london son (51827, 9)


##### **Objetivo 3**
Corroborar que el dataset no cuenta con valores nulos, si los hubiera llenarlos todos con 0.

In [43]:
# Verifico la existencia de Nans
total_nans = london.isnull().sum().sort_values(ascending=False)
if list(total_nans) != 0:
    print("No hay Nans en el dataset")

No hay Nans en el dataset


##### **Objetivo 4**
Mostrar el tipo de dato para las variables "neighbourhood" y "room_type".

In [44]:
# Muestro el tipo de dato para cada variable
neighbourhood_dtype = london["neighbourhood"].dtype
room_type_dtype = london["room_type"].dtype
print(f"Tipo de dato para la variable neighbourhood: {neighbourhood_dtype}")
print(f"Tipo de dato para la variable room_type: {room_type_dtype}")
print(f"{neighbourhood_dtype} significa “objeto”, que en el mundo de Pandas es una secuencia de caracteres (texto)")

Tipo de dato para la variable neighbourhood: object
Tipo de dato para la variable room_type: object
object significa “objeto”, que en el mundo de Pandas es una secuencia de caracteres (texto)


##### **Objetivo 5**
Generar las variables dummies para estas dos variables categóricas y unirlas al dataset london.


In [45]:
# Creo una lista de cada variable para generar luego sus dummies
neigh_list = list(london["neighbourhood"])
room_list = list(london["room_type"])
# Genero las dummies para la variable neighbourhood, dentro de la variable neigh_dummies.
le1 = preprocessing.LabelEncoder()
neigh_dummies = le1.fit_transform(neigh_list)

Imprimir las dimensiones y primeras lineas de estas dummies

In [46]:
# Imprimo el tipo y las dimesiones
print(f"{type(neigh_dummies)} con {np.shape(neigh_dummies)[0]} filas y 0 columnas")

<class 'numpy.ndarray'> con 51827 filas y 0 columnas


In [47]:
# Primeras líneas
neigh_dummies[0:10]

array([13,  8, 18, 29,  1, 18, 18, 29,  1, 27], dtype=int64)

In [48]:
# Genero las dummies para la variable room_type, dentro de la variable room_dummies.
le2 = preprocessing.LabelEncoder()
room_dummies = le2.fit_transform(room_list)

Imprimir las dimensiones y primeras lineas de estas dummies

In [49]:
# Imprimo el tipo y las dimesiones
print(f"{type(room_dummies)} con {np.shape(room_dummies)[0]} filas y 0 columnas")

<class 'numpy.ndarray'> con 51827 filas y 0 columnas


In [50]:
# Primeras líneas
room_dummies[0:10]

array([1, 1, 1, 0, 1, 0, 0, 1, 0, 1], dtype=int64)

Joinear las variables dummies al dataset



In [51]:
# Creo los DataFrame a partir de los np.arrays para poder concatenarlos con .concat()
neigh_dummies_df = pd.DataFrame([neigh_dummies]).T
room_dummies_df = pd.DataFrame([room_dummies]).T

In [52]:
# Cambio el nombre de la columna "0" por neigh_dummies y room_dummies respectivamente en cada DataFrame
neigh_dummies_df = neigh_dummies_df.rename({0 : "neigh_dummies"}, axis = 1)
room_dummies_df = room_dummies_df.rename({0 : "room_dummies"}, axis = 1)

In [53]:
# Concateno las dos variables dummies al dataset principal
london = pd.concat([london,neigh_dummies_df,room_dummies_df], axis = 1)

##### Dividiremos nuestro dataset en la variable dependiente "y" (label o etiqueta) y las variables independientes "x"

In [54]:
# La variable dependiente es el precio, miestras que las independeientes son las otras
# Eliminamos también 'neighbourhood' y 'room_type' porque ya joinié sus dummies
y = np.array(london[["price"]])
x = london.drop(['price', 'neighbourhood', 'room_type'], axis=1)

##### **Objetivo 7**

Dividir el dataset entre Train y Test, tomar un test_size del 30%.

In [55]:
# Separo train y test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=4)

##### **Objetivo 8**
Con el StandardScaler generado en la linea siguiente, realizar el transform para el xtrain y xtest guardandolos en nuevas variables llamadas "xtrain_scal" y "xtest_scal".


In [56]:
# Estandarización de x_train
scaler1 = preprocessing.StandardScaler().fit(x_train)
xtrain_scal = scaler1.transform(x_train)

In [57]:
# Estandarización de x_test
scaler2 = preprocessing.StandardScaler().fit(x_test)
xtest_scal = scaler2.transform(x_test)

##### **Objetivo 9**
Crear un modelo lineal y entrenarlo para nuestro dataset.

Obtener las predicciones y el "mean_squared_error" para el test set.

In [58]:
# Entrenamos el modelo
reg = LinearRegression().fit(xtrain_scal, y_train)

In [59]:
# Predigo un array y_pred con valores aprendidos por el modelo mediante x_test
y_pred = reg.predict(xtest_scal)

In [60]:
# Calculo MSE
mean_squared_error(y_test, y_pred)

1753.243976575432

##### **Objetivo 10**
Crear un SVM Regressor y un GridSearch para realizar la búsqueda de hiperparámetros.

Asignar dos posibles valores para el parámetro "kernel", y tres posibles valores para "C" y "gamma".

Setear al parámetro "refit" en True y hacer 5 CrossValidation.

In [ ]:
# Entreno el modelo
svreg = SVR()
param_svreg = {'kernel':('linear', 'rbf'), 'C':[0.1, 1, 10 ], 'gamma':[0.01, 0.1, 1]}

In [ ]:
regressor_svr = GridSearchCV(svreg, param_svreg, cv = 5, refit = True, verbose = 3)

In [ ]:
regressor_svr.fit(xtrain_scal, y_train.ravel())

##### **Objetivo 11**

Mostrar cuales fueron los mejores hiperparámetros y para estos obtener el mean_squared_error en el test set.

In [ ]:
# Mejores hiperparametros
regressor_svr.best_params_

In [ ]:
# Predigo un array y_pred2 con valores aprendidos por el modelo SVR mediante x_test
y_pred2 = regressor_svr.predict(xtest_scal)

In [ ]:
# Calculo el MSE
mse1 = mean_squared_error(y_test, y_pred2)

##### **Objetivo 12**
Repetir el objetivo 10 pero ahora seteando nuevos valores para los hiperparámetros mencionados con el objetivo de mejorar las predicciones.

Cuales son los mejores hiperparámetros ahora?
Obtenemos una mejora en las predicciones?
Mostrar los resultados.

In [ ]:
# Entreno el modelo
svreg1 = SVR()
param_svreg1 = {'kernel':('linear', 'rbf'), 'C':[0.01, 100, 1000 ], 'gamma':[ 0.0001, 0.001, 10]}
regressor_svr1 = GridSearchCV(svreg1, param_svreg, cv = 5, refit = True, verbose = 3)
regressor_svr1.fit(xtrain_scal, y_train.ravel())

In [ ]:
# Mejores hipermparametros
regressor_svr1.best_params_

In [ ]:
# Predigo un array y_pred3 con valores aprendidos por el modelo SVR mediante x_test
y_pred3 = regressor_svr1.predict(xtest_scal)

In [ ]:
# Calculo el MSE
mse2 = mean_squared_error(y_test, y_pred3)

In [ ]:
# Contrasto los dos MSE obtenidos para comparar el desempeño de los dos modelos
if mse1 > mse2:
    print("No se obtuvo una mejora en las predicciones")
else:
    print("Se obtuvo una mejora en las predicciones")